# **차량 공유업체의 차량 파손 여부 분류하기**

## 0.미션

* 1) 미션1 : Data Preprocessing
    - **과제 수행 목표**
        - 본인의 구글 드라이브에 모델링 수행을 위해 적절한 폴더 및 파일로 **일관성 있게 정리**해야 합니다.
        - 제공된 데이터 : Car_Images.zip
            * Car_Images : 차량의 정상/파손 이미지 무작위 수집

* 2) 미션2 : CNN 모델링
    - **과제 수행 목표**
        - Tensorflow Keras를 이용하여 모델을 3개 이상 생성하세요.
            - 모델 구조와 파라미터는 자유롭게 구성하세요.
            - 단, 세부 목차에서 명시한 부분은 지켜주세요.

* 3) 미션3 : Data Argumentation & Transfer Learning
    - **과제 수행 목표**
        - 성능 개선을 위해 다음의 두가지를 시도하세요.
            * Data Augmentation을 적용하세요.(Image Generator)
            * Transfer Learning(VGG16)


## 1.환경설정 

### (1) 데이터셋 폴더 생성
- **세부요구사항**
    - C드라이브에 Datasets라는 폴더를 만드세요.
        - 구글드라이브를 사용하는경우 드라이브 첫 화면에 Datasets 라는 폴더를 만드세요. ('/content/drive/MyDrive/Datasets/')
    - 해당 폴더 안에 Car_Images.zip 파일을 넣으세요.

* 구글 Colab을 이용하는 경우

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### (2) 데이터셋 불러오기 
- **세부요구사항**
    - Car_Images.zip 파일을 C:/Datasets/ 경로에 압축 해제합니다.
    - zipfile 모듈을 이용하거나 다른 방식을 사용해도 됩니다.
        - 참고 자료 : [zipfile document](https://docs.python.org/3/library/zipfile.html#zipfile-objects)
    - 폴더구조(로컬)
        * C:/Datasets/ : 압축파일
        * C:/Datasets/Car_Images_train/ : 압축 해제한 이미지 저장소
    - 폴더구조(구글드라이브브)
        * /content/drive/MyDrive/Datasets/ : 압축파일
        * /content/drive/MyDrive/Datasets/Car_Images_train/ : 압축 해제한 이미지 저장소
    - 압축을 해제하면 다음과 같은 두 하위 폴더가 생성됩니다.
        * normal, abnormal : 각 폴더에는 이미지들이 있습니다.
        * 이후 단계에서 해당 경로로 부터 validation, test 셋을 추출하게 됩니다.
        

In [ ]:
import zipfile
import os, numpy, pandas

In [ ]:
# 압축파일 경로
# 구글 드라이브인 경우 경로에 맞게 지정하세요.
dataset_path  = '/content/drive/MyDrive/Datasets/'
# dataset_path = 'C:/Datasets/'

file_path = dataset_path + 'Car_Images.zip'

In [ ]:
# 압축 해제
# data = zipfile.ZipFile(file_path)
# data.extractall('경로')
!unzip /content/drive/MyDrive/Datasets/Car_Images.zip -d /content/drive/MyDrive/Datasets/car_image

### (3) 이미지 저장을 위한 폴더 생성
- **세부요구사항**
    - train, validation, test 을 위해 각각 하위 폴더 normal과 abnormal를 준비합니다.
        - train
            * 정상 이미지 저장소 : C:/Datasets/Car_Images_train/normal/ 
                * 구글드라이브 :   /content/drive/MyDrive/Datasets/Car_Images_train/normal/
            * 파손 이미지 저장소 : C:/Datasets/Car_Images_train/abnormal/
                * 구글드라이브 : /content/drive/MyDrive/Datasets/Car_Images_train/abnormal/
        - val, test 역시 동일한 구조로 생성합니다.
    - 직접 탐색기에서 폴더를 생성할 수도 있고, os 모듈을 이용하여 코드로 작성할 수도 있습니다.
        - 참고 자료 : [os document](https://docs.python.org/3/library/os.html)

In [ ]:
!mkdir /content/drive/MyDrive/Datasets/Car_Images_train
!mkdir /content/drive/MyDrive/Datasets/Car_Images_validation
!mkdir /content/drive/MyDrive/Datasets/Car_Images_test

In [ ]:
! mkdir /content/drive/MyDrive/Datasets/Car_Images_train/normal; mkdir /content/drive/MyDrive/Datasets/Car_Images_train/abnormal
! mkdir /content/drive/MyDrive/Datasets/Car_Images_validation/normal; mkdir /content/drive/MyDrive/Datasets/Car_Images_validation/abnormal
! mkdir /content/drive/MyDrive/Datasets/Car_Images_test/normal; mkdir /content/drive/MyDrive/Datasets/Car_Images_test/abnormal

In [ ]:

# 각각 경로 지정



# train 폴더는 압축을 해제하면서 이미 생성 되어 있습니다.

# test 폴더 만들기 os.mkdir()



# validation 폴더 만들기



## 2.데이터 전처리

### (1) 데이터 분할 : Training set | Validation set | Test set 생성
- **세부요구사항**
    - Training set, Validation set, Test set을 만듭니다.
        * size
            * test : 전체에서 20%를 추출합니다.
            * validation : test를 떼어낸 나머지에서 다시 20%를 추출합니다.
        * 데이터는 랜덤하게 추출해야 합니다.
            - random, shutil 모듈을 이용하여 랜덤하게 추출할 수 있습니다.
                - [random document](https://docs.python.org/3/library/random.html) | [shutil document](https://docs.python.org/3/library/shutil.html)
            * 해당 모듈 이외에 자신이 잘 알고 있는 방법을 사용해도 됩니다.
---

#### 1) test, validation 크기를 지정

In [ ]:
import random, shutil, os

In [ ]:
import sklearn

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# 전체 이미지 갯수를 확인합니다.
ab_image_path = '/content/drive/MyDrive/Datasets/car_image/abnormal'
nor_image_path = '/content/drive/MyDrive/Datasets/car_image/normal'
len(os.listdir('/content/drive/MyDrive/Datasets/car_image/normal')) , len(os.listdir('/content/drive/MyDrive/Datasets/car_image/abnormal'))

(302, 303)

In [ ]:
test_ab_dir = '/content/drive/MyDrive/Datasets/Car_Images_test/abnormal'
val_ab_dir = '/content/drive/MyDrive/Datasets/Car_Images_validation/abnormal'
train_ab_dir = '/content/drive/MyDrive/Datasets/Car_Images_train/abnormal'
test_nor_dir = '/content/drive/MyDrive/Datasets/Car_Images_test/normal'
val_nor_dir = '/content/drive/MyDrive/Datasets/Car_Images_validation/normal'
train_nor_dir = '/content/drive/MyDrive/Datasets/Car_Images_train/normal'

In [ ]:
# normal

# 원본 이미지 경로
nor_dir = '/content/drive/MyDrive/Datasets/car_image/normal'

# 분할된 이미지를 저장할 폴더 경로
test_nor_dir = '/content/drive/MyDrive/Datasets/Car_Images_test/normal'
val_nor_dir = '/content/drive/MyDrive/Datasets/Car_Images_validation/normal'
train_nor_dir = '/content/drive/MyDrive/Datasets/Car_Images_train/normal'

# 이미지 파일 목록 가져오기
files = os.listdir(nor_dir)

train_val_files, test_files = train_test_split(files, test_size=0.1, random_state=2023)
train_files, val_files = train_test_split(train_val_files, test_size=0.2, random_state=2023)

# test set 복사
for file in test_files:
    src_file = os.path.join(nor_dir, file)
    dst_file = os.path.join(test_nor_dir, file)
    shutil.copy(src_file, dst_file)

# validation set 복사
for file in val_files:
    src_file = os.path.join(nor_dir, file)
    dst_file = os.path.join(val_nor_dir, file)
    shutil.copy(src_file, dst_file)

# train set 복사
for file in train_files:
    src_file = os.path.join(nor_dir, file)
    dst_file = os.path.join(train_nor_dir, file)
    shutil.copy(src_file, dst_file)

In [ ]:
# abnormal

# 원본 이미지 경로
ab_dir = '/content/drive/MyDrive/Datasets/car_image/abnormal'

# 분할된 이미지를 저장할 폴더 경로
test_ab_dir = '/content/drive/MyDrive/Datasets/Car_Images_test/abnormal'
val_ab_dir = '/content/drive/MyDrive/Datasets/Car_Images_validation/abnormal'
train_ab_dir = '/content/drive/MyDrive/Datasets/Car_Images_train/abnormal'

# 이미지 파일 목록 가져오기
files = os.listdir(ab_dir)

train_val_files, test_files = train_test_split(files, test_size=0.1, random_state=2023)
train_files, val_files = train_test_split(train_val_files, test_size=0.2, random_state=2023)

# test set 복사
for file in test_files:
    src_file = os.path.join(ab_dir, file)
    dst_file = os.path.join(test_ab_dir, file)
    shutil.copy(src_file, dst_file)

# validation set 복사
for file in val_files:
    src_file = os.path.join(ab_dir, file)
    dst_file = os.path.join(val_ab_dir, file)
    shutil.copy(src_file, dst_file)

# train set 복사
for file in train_files:
    src_file = os.path.join(ab_dir, file)
    dst_file = os.path.join(train_ab_dir, file)
    shutil.copy(src_file, dst_file)

In [ ]:
len(os.listdir('/content/drive/MyDrive/Datasets/Car_Images_test/normal')), len(os.listdir('/content/drive/MyDrive/Datasets/Car_Images_train/normal')), len(os.listdir('/content/drive/MyDrive/Datasets/Car_Images_validation/normal'))

(31, 216, 55)

In [ ]:
len(os.listdir('/content/drive/MyDrive/Datasets/Car_Images_test/abnormal')), len(os.listdir('/content/drive/MyDrive/Datasets/Car_Images_train/abnormal')), len(os.listdir('/content/drive/MyDrive/Datasets/Car_Images_validation/abnormal'))

(31, 217, 55)

In [ ]:
# 추출 후 데이터 크기
print(f'test: {len(os.listdir(test_ab_dir)), len(os.listdir(test_nor_dir))}')
print(f'val: {len(os.listdir(val_ab_dir)), len(os.listdir(val_nor_dir))}')
print(f'train: {len(os.listdir(train_ab_dir)), len(os.listdir(train_nor_dir))}')

test: (31, 31)
val: (55, 55)
train: (217, 216)


In [ ]:
# # test 사이즈 : 전체 이미지의 20%
# te_data_num = [round(len(os.listdir(nor_image_path))*0.1), round(len(os.listdir(ab_image_path))*0.1)]
# print(te_data_num)

# # validation 사이즈 : test를 제외한 나머지 중에서 20%
# val_data_num = [ round((len(os.listdir(nor_image_path))-te_data_num[0])*0.2) , round((len(os.listdir(nor_image_path))-te_data_num[1])*0.2) ]
# print(val_data_num)

# # train 사이즈
# train_data_num = [len(os.listdir(nor_image_path)) - te_data_num[0] - val_data_num[0],
#                   len(os.listdir(ab_image_path))- te_data_num[1] - val_data_num[1]]

#### 2) test 셋 추출

In [ ]:
random.seed(2023)









In [ ]:
# 추출 후 이미지 갯수 확인




#### 3) validation 셋 추출

In [ ]:
random.seed(2023)







In [ ]:
# 추출 후 이미지 갯수 확인




### (2) 데이터 복사 및 이동
- **세부요구사항**
    - 분할된 데이터를 복사 이동합니다.
        - 새로운 폴더에 저장하는 데이터로 "3.모델링I"에서 사용합니다.
        - 기존 폴더는 "4.모델링II > (1) Data Augmentation"에서 사용합니다.
    - Training set | Validation set | Test set의 데이터를 **새로운 폴더**에 복사하세요.
        - 새로운 폴더 명
            * copy_images/trainset
            * copy_images/validset
            * copy_images/testset
        - 새로운 폴더에는 normal, abnormal 파일 모두를 복사합니다. 
            * 파일을 구분하기 위해 abnormal 파일들은 파일명 앞에 접두사 'ab_'를 붙입시다.
        - os, shutil 모듈을 활용하세요.

In [ ]:
# copy 폴더 생성
!mkdir /content/drive/MyDrive/Datasets/copy_images

In [ ]:
# !mkdir /content/drive/MyDrive/Datasets/copy_images/trainset
# !mkdir /content/drive/MyDrive/Datasets/copy_images/validset
# !mkdir /content/drive/MyDrive/Datasets/copy_images/testset

In [ ]:
# !mkdir /content/drive/MyDrive/Datasets/copy_images/trainset/normal; mkdir /content/drive/MyDrive/Datasets/copy_images/trainset/abnormal
# !mkdir /content/drive/MyDrive/Datasets/copy_images/validset/normal; mkdir /content/drive/MyDrive/Datasets/copy_images/validset/abnormal
# !mkdir /content/drive/MyDrive/Datasets/copy_images/testset/normal; mkdir /content/drive/MyDrive/Datasets/copy_images/testset/abnormal

#### 1) abnormal 파일 복사

* 복사하기 : shutil.copytree()

In [ ]:
# 주의 shutil.copytree()를 사용할 때 해당 경로에 같은 이름의 파일이 있으면 오류남;; 경로에 파일 미리 만들기 X

copy_test_ab_dir = '/content/drive/MyDrive/Datasets/copy_images/testset'
copy_val_ab_dir = '/content/drive/MyDrive/Datasets/copy_images/validset'
copy_train_ab_dir = '/content/drive/MyDrive/Datasets/copy_images/trainset'

shutil.copytree(test_ab_dir, copy_test_ab_dir)
shutil.copytree(val_ab_dir, copy_val_ab_dir)
shutil.copytree(train_ab_dir, copy_train_ab_dir)


* abnormal 이미지 이름의 접두어 "ab_" 붙이기 : os.rename

In [ ]:
import random

In [ ]:
# copy_image_testset의 abnormal 이름 변경
for filename in os.listdir(copy_test_ab_dir):
    rename = 'ab_' + filename
    os.rename(os.path.join(copy_test_ab_dir, filename), os.path.join(copy_test_ab_dir, rename))

In [ ]:
# copy_image_validset의 abnormal 이름 변경
for filename in os.listdir(copy_val_ab_dir):
    rename = 'ab_' + filename
    os.rename(os.path.join(copy_val_ab_dir, filename), os.path.join(copy_val_ab_dir, rename))

In [ ]:
# copy_image_trainset의 abnormal 이름 변경
for filename in os.listdir(copy_train_ab_dir):
    rename = 'ab_' + filename
    os.rename(os.path.join(copy_train_ab_dir, filename), os.path.join(copy_train_ab_dir, rename))

In [ ]:
# 이동동 후 데이터 크기
print(f'copy_test: {len(os.listdir(copy_test_ab_dir))}')
print(f'copy_val: {len(os.listdir(copy_val_ab_dir))}')
print(f'copy_train: {len(os.listdir(copy_train_ab_dir))}')

NameError: ignored

#### 2) normal 파일 복사

In [ ]:
# test 파일을 이동
# 복사사할 파일 리스트를 files에 저장
files = os.listdir(test_nor_dir)

for file in files:
    source_path = os.path.join(test_nor_dir, file)
    target_path = os.path.join(copy_test_nor_dir, file)
    shutil.copy(source_path, target_path)

# val 파일을 이동
# 복사사할 파일 리스트를 files에 저장
files = os.listdir(val_nor_dir)

for file in files:
    source_path = os.path.join(val_nor_dir, file)
    target_path = os.path.join(copy_val_nor_dir, file)
    shutil.copy(source_path, target_path)

# train 파일을 이동
# 복사사할 파일 리스트를 files에 저장
files = os.listdir(train_nor_dir)

for file in files:
    source_path = os.path.join(train_nor_dir, file)
    target_path = os.path.join(copy_train_nor_dir, file)
    shutil.copy(source_path, target_path)


In [ ]:
a = os.listdir('/content/drive/MyDrive/Datasets/copy_images/testset')
b = os.listdir('/content/drive/MyDrive/Datasets/copy_images/trainset')
c = os.listdir('/content/drive/MyDrive/Datasets/copy_images/validset')


In [ ]:
random.seed(2023)
random.shuffle(a)
random.shuffle(b)
random.shuffle(c)

In [ ]:
a[:3]

['DALLбдE 2023-03-10 18.52.13 - photo of a part of car.png',
 'ab_DALLбдE 2023-03-11 01.25.20 - slightly scratched car.png',
 'ab_DALLбдE 2023-03-11 18.46.21 - scratched car.png']

* 데이터 갯수 조회

In [ ]:
print('train:', len(os.listdir(dataset_path+'copy_images/trainset/')))
print('valid:', len(os.listdir(dataset_path+'copy_images/validset/')))
print('test:', len(os.listdir(dataset_path+'copy_images/testset/')))

NameError: ignored

## 3.모델링 I
* **세부요구사항**
    * 모델링을 위한 데이터 구조 만들기
        * x : 이미지를 array로 변환합니다.
        * y : 이미지 갯수만큼 normal - 0, abnormal - 1 로 array를 만듭니다.
    * 모델을 최소 3개 이상 만들고 성능을 비교합니다.
        * 모델 학습 과정에 알맞은 보조 지표를 사용하세요.
        * 전처리 과정에서 생성한 Validation set을 적절하게 사용하세요.
        * Early Stopping을 반드시 사용하세요.
            * 최적의 가중치를 모델에 적용하세요.

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report, confusion_matrix

### (1) X : image to array
- **세부요구사항**
    * 모델링을 위해서는 np.array 형태로 데이터셋을 만들어야 합니다.
    * Training set / Validation set / Test set의 X는 이미지 형태로 되어있습니다. 
    * 이미지 파일을 불러와 train, valid, test 각각 array 형태로 변환해 봅시다.
        * 각 폴더로 부터 이미지 목록을 만들고
        * 이미지 한장씩 적절한 크기로 로딩하여 (keras.utils.load_img)
            * 이미지가 너무 크면 학습시간이 많이 걸리고, 메모리 부족현상이 발생될 수 있습니다.
            * 이미지 크기를 280 * 280 * 3 이내의 크기를 설정하여 로딩하시오.
            * array로 변환 (keras.utils.img_to_array, np.expand_dims)
        * 데이터셋에 추가합니다.(데이터셋도 array)

#### 1) 이미지 목록 만들기
* train, validation, test 폴더로 부터 이미지 목록을 생성합니다.

In [ ]:
# # 이미지 목록 저장
img_train_list = os.listdir(dataset_path+'copy_images/trainset/')
img_valid_list = os.listdir(dataset_path+'copy_images/validset/')
img_test_list = os.listdir(dataset_path+'copy_images/testset/')

In [ ]:
# # 메모리, 처리시간을 위해서 이미지 크기 조정
img_size = 280 ## 사이즈 조정 가능

#### 2) 이미지들을 배열 데이터셋으로 만들기

In [ ]:
img_train_list

['ab_DALLбдE 2023-03-11 15.13.48 - dents of a car.png',
 'ab_DALLбдE 2023-03-11 01.32.25 - a little bit scratched car.png',
 'ab_DALLбдE 2023-03-10 23.28.30 - photo of a part of car without blemish.png',
 'ab_DALLбдE 2023-03-11 17.28.29 - slightly dented car.png',
 'ab_DALLбдE 2023-03-10 22.37.56 - photo of a part of car.png',
 'ab_DALLбдE 2023-03-11 14.59.32 - dents of a car.png',
 'ab_DALLбдE 2023-03-11 00.12.34 - a part of car without scrach.png',
 'ab_DALLбдE 2023-03-11 17.16.50 - damaged car.png',
 'ab_DALLбдE 2023-03-11 18.42.01 - slightly dented car.png',
 'ab_DALLбдE 2023-03-11 15.07.10 - dents of a car.png',
 'ab_DALLбдE 2023-03-11 18.41.54 - slightly dented car.png',
 'ab_DALLбдE 2023-03-11 18.46.26 - scratched car.png',
 'ab_DALLбдE 2023-03-11 01.29.53 - a little bit scratched car.png',
 'ab_DALLбдE 2023-03-11 18.43.01 - slightly dented car.png',
 'ab_DALLбдE 2023-03-11 15.12.25 - dents of a car.png',
 'ab_DALLбдE 2023-03-11 01.21.50 - scratched car.png',
 'ab_DALLбдE 2023-0

In [ ]:
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.inception_v3 import preprocess_input
import numpy as np
import glob


In [ ]:
img_train_list

In [ ]:
copy_files_train = glob.glob('/content/drive/MyDrive/Datasets/copy_images/trainset/*')
train_path = '/content/drive/MyDrive/Datasets/copy_images/trainset/'
images = []
for file in img_train_list:
    img = image.load_img(train_path + file, target_size=(280,280))
    img = image.img_to_array(img)
    img = preprocess_input(img)
    images.append(img)
x_train = np.array(images)

In [ ]:
copy_files_val = glob.glob('/content/drive/MyDrive/Datasets/copy_images/validset/*')
val_path = '/content/drive/MyDrive/Datasets/copy_images/validset/'
images = []
for file in img_valid_list:
    img = image.load_img(val_path + file, target_size=(280,280))
    img = image.img_to_array(img)
    img = preprocess_input(img)
    images.append(img)
x_val = np.array(images)

In [ ]:
copy_files_test = glob.glob('/content/drive/MyDrive/Datasets/copy_images/testset/*')
test_path = '/content/drive/MyDrive/Datasets/copy_images/testset/'
images = []
for file in img_test_list:
    img = image.load_img(test_path + file, target_size=(280,280))
    img = image.img_to_array(img)
    img = preprocess_input(img)
    images.append(img)
x_test = np.array(images)

In [ ]:
x_train.shape, x_val.shape, x_test.shape

((433, 280, 280, 3), (110, 280, 280, 3), (62, 280, 280, 3))

### (2) y : 클래스 만들기
- **세부요구사항**
    - Training set / Validation set / Test set의 y를 생성합니다.
        - 각각 normal, abnormal 데이터의 갯수를 다시 확인하고
        - normal을 0, abnormal을 1로 지정합니다.

In [ ]:
# 데이터 갯수 확인
print( len(img_train_list) )
print( len([val for val in img_train_list if val.startswith('ab_')]) )
print('---')
print( len(img_valid_list) )
print( len([val for val in img_valid_list if val.startswith('ab_')]) )
print('---')
print( len(img_test_list) )
print( len([val for val in img_test_list if val.startswith('ab_')]) )

433
217
---
110
55
---
62
31


* y_train, y_valid, y_test 만들기
    * normal, abnormal 데이터의 갯수를 다시 확인하고 normal을 0, abnormal을 1로 지정합니다.

In [ ]:
copy_files_train[:10]

['/content/drive/MyDrive/Datasets/copy_images/trainset/ab_DALLбдE 2023-03-11 15.13.48 - dents of a car.png',
 '/content/drive/MyDrive/Datasets/copy_images/trainset/ab_DALLбдE 2023-03-11 01.32.25 - a little bit scratched car.png',
 '/content/drive/MyDrive/Datasets/copy_images/trainset/ab_DALLбдE 2023-03-10 23.28.30 - photo of a part of car without blemish.png',
 '/content/drive/MyDrive/Datasets/copy_images/trainset/ab_DALLбдE 2023-03-11 17.28.29 - slightly dented car.png',
 '/content/drive/MyDrive/Datasets/copy_images/trainset/ab_DALLбдE 2023-03-10 22.37.56 - photo of a part of car.png',
 '/content/drive/MyDrive/Datasets/copy_images/trainset/ab_DALLбдE 2023-03-11 14.59.32 - dents of a car.png',
 '/content/drive/MyDrive/Datasets/copy_images/trainset/ab_DALLбдE 2023-03-11 00.12.34 - a part of car without scrach.png',
 '/content/drive/MyDrive/Datasets/copy_images/trainset/ab_DALLбдE 2023-03-11 17.16.50 - damaged car.png',
 '/content/drive/MyDrive/Datasets/copy_images/trainset/ab_DALLбдE 20

In [ ]:
# y_train
temp = []
for i in img_train_list:
    if i.startswith('ab_'):
        temp.append(1)
    else:
        temp.append(0)
y_train = np.array(temp)

In [ ]:
type(i)

str

In [ ]:
y_train

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [ ]:
# y_valid
temp = []
for i in img_valid_list:
    if i.startswith('ab_'):
        temp.append(1)
    else:
        temp.append(0)
y_val = np.array(temp)

In [ ]:
# y_test
temp = []
for i in img_test_list:
    if i.startswith('ab_'):
        temp.append(1)
    else:
        temp.append(0)
y_test = np.array(temp)

In [ ]:
# Scaling 표준화화
mean_n = x_train.mean()
std_n = x_train.std()

x_train_sc = (x_train - mean_n) / std_n
x_test_sc = (x_test - mean_n) / std_n
x_val_sc = (x_val - mean_n) / std_n

In [ ]:
# Scaling MinMax
x_max, x_min = x_train.max(), x_train.min()
x_train_mi = (x_train                                                                                                                                                                                                                                                                                                                                                                                                                                                                      - x_max)/(x_max - x_min)
x_test_mi = (x_test - x_max)/(x_max - x_min)
x_val_mi = (x_val - x_max)/(x_max - x_min)

In [ ]:
# one-hot Encoding
from tensorflow.keras.utils import to_categorical
class_n = len(np.unique(y_train))
y_train = to_categorical(y_train, class_n)
y_val = to_categorical(y_val, class_n)
y_test = to_categorical(y_test, class_n)

array([1.], dtype=float32)

In [ ]:
y_train.shape, y_val.shape, y_test.shape

((433, 2), (110, 2), (62, 2))

### (3) 모델1
- **세부요구사항**
    - Conv2D, MaxPooling2D, Flatten, Dense 레이어들을 이용하여 모델을 설계
    - 학습시 validation_data로 validation set을 사용하시오.
    - 반드시 Early Stopping 적용
    - 평가시, confusion matrix, accuracy, recall, precision, f1 score 등을 이용하시오.

In [ ]:
x_train.shape, y_train.shape

((433, 280, 280, 3), (433, 2))

#### 1) 구조 설계

In [ ]:
# Functional API

# 라이브러리
import tensorflow as tf
from tensorflow import keras

# 세션클리어
keras.backend.clear_session()

# layer 연결
il = keras.layers.Input(shape=(280, 280, 3))

hl = keras.layers.Conv2D(filters=32, kernel_size=(3,3), strides=(1,1), activation='relu')(il)
hl = keras.layers.Conv2D(filters=32, kernel_size=(3,3), strides=(1,1), activation='relu')(hl)
hl = keras.layers.Conv2D(filters=32, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu')(hl)
hl = keras.layers.MaxPool2D(pool_size=(2,2))(hl)
hl = keras.layers.BatchNormalization()(hl)

hl = keras.layers.Conv2D(filters=64, kernel_size=(3,3), strides=(1,1), activation='relu')(hl)
hl = keras.layers.Conv2D(filters=64, kernel_size=(3,3), strides=(1,1), activation='relu')(hl)
hl = keras.layers.Conv2D(filters=64, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu')(hl)
hl = keras.layers.MaxPool2D(pool_size=(2,2))(hl)
hl = keras.layers.BatchNormalization()(hl)

hl = keras.layers.Conv2D(filters=128, kernel_size=(4,4), strides=(1,1), padding='same', activation='relu')(hl)
hl = keras.layers.Conv2D(filters=128, kernel_size=(4,4), strides=(1,1), padding='same', activation='relu')(hl)
hl = keras.layers.Conv2D(filters=128, kernel_size=(4,4), strides=(1,1), padding='same', activation='relu')(hl)
hl = keras.layers.MaxPool2D(pool_size=(2,2))(hl)
hl = keras.layers.BatchNormalization()(hl)

hl = keras.layers.Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu')(hl)
hl = keras.layers.Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu')(hl)
hl = keras.layers.Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu')(hl)
hl = keras.layers.MaxPool2D(pool_size=(2,2))(hl)
hl = keras.layers.BatchNormalization()(hl)

hl = keras.layers.Conv2D(filters=128, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu')(hl)
hl = keras.layers.Conv2D(filters=128, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu')(hl)
hl = keras.layers.Conv2D(filters=128, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu')(hl)
hl = keras.layers.MaxPool2D(pool_size=(2,2))(hl)
hl = keras.layers.BatchNormalization()(hl)

hl = keras.layers.Flatten()(hl)

hl = keras.layers.Dense(64, activation='relu')(hl)
hl = keras.layers.Dense(32, activation='relu')(hl)
hl = keras.layers.Dropout(0.3)(hl)
ol = keras.layers.Dense(2, activation='sigmoid')(hl)

# 모델 생성
model = keras.Model(il, ol)

In [ ]:
from tensorflow.keras.layers import *

In [ ]:
# 모델 컴파일
model.compile(loss=keras.losses.binary_crossentropy, metrics=['accuracy'], optimizer='adam')

# 요약
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 280, 280, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 278, 278, 32)      896       
                                                                 
 conv2d_1 (Conv2D)           (None, 276, 276, 32)      9248      
                                                                 
 conv2d_2 (Conv2D)           (None, 276, 276, 32)      9248      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 138, 138, 32)     0         
 )                                                               
                                                                 
 batch_normalization (BatchN  (None, 138, 138, 32)     128       
 ormalization)                                               

#### 2) 학습
* EarlyStopping 설정하고 학습시키기

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
es = EarlyStopping(monitor='val_loss', min_delta=0, patience=15, verbose=1, restore_best_weights=True)

In [ ]:
hist = model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=10000, verbose=1, callbacks=[es])

Epoch 1/10000
14/14 [==============================] - 35s 982ms/step - loss: 0.6871 - accuracy: 0.7021 - val_loss: 1.2007 - val_accuracy: 0.5000
Epoch 2/10000
14/14 [==============================] - 6s 454ms/step - loss: 0.5178 - accuracy: 0.8060 - val_loss: 0.6645 - val_accuracy: 0.6273
Epoch 3/10000
14/14 [==============================] - 7s 476ms/step - loss: 0.4430 - accuracy: 0.8383 - val_loss: 1.2135 - val_accuracy: 0.4818
Epoch 4/10000
14/14 [==============================] - 7s 470ms/step - loss: 0.4339 - accuracy: 0.8545 - val_loss: 0.8682 - val_accuracy: 0.6182
Epoch 5/10000
14/14 [==============================] - 6s 451ms/step - loss: 0.3318 - accuracy: 0.8637 - val_loss: 1.6990 - val_accuracy: 0.6182
Epoch 6/10000
14/14 [==============================] - 6s 464ms/step - loss: 0.3528 - accuracy: 0.8406 - val_loss: 0.8664 - val_accuracy: 0.4727
Epoch 7/10000
14/14 [==============================] - 6s 462ms/step - loss: 0.3363 - accuracy: 0.8614 - val_loss: 0.8094 - val_a

#### 3) test set으로 예측하고 평가하기
* 평가는 confusion_matrix, classification_report 활용

In [ ]:
y_pred = model.predict(x_test_sc)

2/2 [==============================] - 3s 2s/step


In [ ]:
y_pred.shape

(62, 2)

In [ ]:
y_pred_new = y_pred.argmax(axis=1)
y_pred_new.shape

(62,)

In [ ]:
y_test.shape

(62, 2)

In [ ]:
y_test_new = y_test.argmax(axis=1)
y_test_new.shape

(62,)

In [ ]:
print(f'{classification_report(y_test_new, y_pred_new)}')

              precision    recall  f1-score   support

           0       0.57      0.68      0.62        31
           1       0.60      0.48      0.54        31

    accuracy                           0.58        62
   macro avg       0.58      0.58      0.58        62
weighted avg       0.58      0.58      0.58        62



### (4) 모델2
- **세부요구사항**
    - Conv2D, MaxPooling2D, Flatten, Dense 레이어들을 이용하여 모델을 설계
    - 학습시 validation_data로 validation set을 사용하시오.
    - 반드시 Early Stopping 적용
    - 평가시, confusion matrix, accuracy, recall, precision, f1 score 등을 이용하시오.

#### 1) 구조 설계

In [ ]:
# Functional API

# 라이브러리
import tensorflow as tf
from tensorflow import keras

# 세션클리어
keras.backend.clear_session()

# layer 연결
il = keras.layers.Input(shape=(280, 280, 3))

hl = keras.layers.Conv2D(filters=64, kernel_size=(5,5), strides=(1,1), padding='same', activation='relu')(il)
hl = keras.layers.MaxPool2D(pool_size=(2,2))(hl)

hl = keras.layers.Conv2D(filters=64, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu')(hl)
hl = keras.layers.Conv2D(filters=64, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu')(hl)
hl = keras.layers.Conv2D(filters=64, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu')(hl)
hl = keras.layers.Conv2D(filters=64, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu')(hl)
hl = keras.layers.MaxPool2D(pool_size=(2,2))(hl)
hl = keras.layers.BatchNormalization()(hl)

hl = keras.layers.Conv2D(filters=128, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu')(hl)
hl = keras.layers.Conv2D(filters=128, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu')(hl)
hl = keras.layers.Conv2D(filters=128, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu')(hl)
hl = keras.layers.MaxPool2D(pool_size=(2,2))(hl)
hl = keras.layers.BatchNormalization()(hl)

hl = keras.layers.Conv2D(filters=128, kernel_size=(4,4), strides=(1,1), activation='relu')(hl)
hl = keras.layers.Conv2D(filters=128, kernel_size=(4,4), strides=(1,1), activation='relu')(hl)
hl = keras.layers.Conv2D(filters=128, kernel_size=(4,4), strides=(1,1), padding='same', activation='relu')(hl)
hl = keras.layers.MaxPool2D(pool_size=(2,2))(hl)
hl = keras.layers.BatchNormalization()(hl)

hl = keras.layers.Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu')(hl)
hl = keras.layers.Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu')(hl)
hl = keras.layers.Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu')(hl)
hl = keras.layers.MaxPool2D(pool_size=(2,2))(hl)
hl = keras.layers.BatchNormalization()(hl)

hl = keras.layers.Conv2D(filters=128, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu')(hl)
hl = keras.layers.Conv2D(filters=128, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu')(hl)
hl = keras.layers.Conv2D(filters=128, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu')(hl)
hl = keras.layers.MaxPool2D(pool_size=(2,2))(hl)
hl = keras.layers.BatchNormalization()(hl)

hl = keras.layers.Flatten()(hl)

hl = keras.layers.Dense(64, activation='relu')(hl)
hl = keras.layers.Dense(32, activation='relu')(hl)
hl = keras.layers.Dropout(0.3)(hl)
ol = keras.layers.Dense(2, activation='sigmoid')(hl)

# 모델 생성
model2 = keras.Model(il, ol)

# 모델 컴파일
model2.compile(loss=keras.losses.binary_crossentropy, metrics=['accuracy'], optimizer='adam')

# 요약
model2.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 280, 280, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 280, 280, 64)      4864      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 140, 140, 64)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 140, 140, 64)      36928     
                                                                 
 conv2d_2 (Conv2D)           (None, 140, 140, 64)      36928     
                                                                 
 conv2d_3 (Conv2D)           (None, 140, 140, 64)      36928     
                                                             

#### 2) 학습
* EarlyStopping 설정하고 학습시키기

In [ ]:
hist2 = model2.fit(x_train_sc, y_train, validation_data=(x_val_sc, y_val), epochs=10000, verbose=1, callbacks=[es])

Epoch 1/10000
14/14 [==============================] - 5s 343ms/step - loss: 0.1074 - accuracy: 0.9654 - val_loss: 8.8294 - val_accuracy: 0.6818
Epoch 2/10000
14/14 [==============================] - 4s 320ms/step - loss: 0.1172 - accuracy: 0.9654 - val_loss: 9.3490 - val_accuracy: 0.6455
Epoch 3/10000
14/14 [==============================] - 5s 347ms/step - loss: 0.1403 - accuracy: 0.9538 - val_loss: 5.2631 - val_accuracy: 0.6273
Epoch 4/10000
14/14 [==============================] - 5s 324ms/step - loss: 0.2260 - accuracy: 0.9053 - val_loss: 14.9209 - val_accuracy: 0.5818
Epoch 5/10000
14/14 [==============================] - 5s 323ms/step - loss: 0.1782 - accuracy: 0.9400 - val_loss: 29.4966 - val_accuracy: 0.5091
Epoch 6/10000
14/14 [==============================] - 5s 348ms/step - loss: 0.2253 - accuracy: 0.9145 - val_loss: 40.2655 - val_accuracy: 0.5909
Epoch 7/10000
14/14 [==============================] - 5s 326ms/step - loss: 0.2297 - accuracy: 0.8938 - val_loss: 22.3192 - va

#### 3) test set으로 예측하고 평가하기
* 평가는 confusion_matrix, classification_report 활용

In [ ]:
y_pred2 = model2.predict(x_test_sc)
y_pred_new2 = y_pred2.argmax(axis=1)
y_test_new2 = y_test.argmax(axis=1)

2/2 [==============================] - 0s 121ms/step


In [ ]:
print(f'{classification_report(y_test_new, y_pred_new2)}')

              precision    recall  f1-score   support

           0       0.75      0.77      0.76        31
           1       0.77      0.74      0.75        31

    accuracy                           0.76        62
   macro avg       0.76      0.76      0.76        62
weighted avg       0.76      0.76      0.76        62



### (5) 모델3
- **세부요구사항**
    - Conv2D, MaxPooling2D, Flatten, Dense 레이어들을 이용하여 모델을 설계
    - 학습시 validation_data로 validation set을 사용하시오.
    - 반드시 Early Stopping 적용
    - 평가시, confusion matrix, accuracy, recall, precision, f1 score 등을 이용하시오.

#### 1) 구조 설계

In [ ]:
# Functional API

# 라이브러리
import tensorflow as tf
from tensorflow import keras

# 세션클리어
keras.backend.clear_session()

# layer 연결
il = keras.layers.Input(shape=(280, 280, 3))

hl = keras.layers.Conv2D(filters=64, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu')(il)
hl = keras.layers.MaxPool2D(pool_size=(2,2))(hl)

hl = keras.layers.Conv2D(filters=64, kernel_size=(4,4), strides=(1,1), padding='same', activation='relu')(hl)
hl = keras.layers.Conv2D(filters=64, kernel_size=(4,4), strides=(1,1), padding='same', activation='relu')(hl)
hl = keras.layers.Conv2D(filters=64, kernel_size=(4,4), strides=(1,1), padding='same', activation='relu')(hl)
hl = keras.layers.MaxPool2D(pool_size=(2,2))(hl)
hl = keras.layers.BatchNormalization()(hl)

hl = keras.layers.Conv2D(filters=128, kernel_size=(4,4), strides=(1,1), padding='same', activation='relu')(hl)
hl = keras.layers.Conv2D(filters=128, kernel_size=(4,4), strides=(1,1), padding='same', activation='relu')(hl)
hl = keras.layers.Conv2D(filters=128, kernel_size=(4,4), strides=(1,1), padding='same', activation='relu')(hl)

hl = keras.layers.Flatten()(hl)

hl = keras.layers.Dense(64, activation='relu')(hl)
hl = keras.layers.Dropout(0.25)(hl)
ol = keras.layers.Dense(2, activation='sigmoid')(hl)

# 모델 생성
model3 = keras.Model(il, ol)

# 모델 컴파일
model3.compile(loss=keras.losses.binary_crossentropy, metrics=['accuracy'], optimizer='adam')

# 요약
model3.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 280, 280, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 280, 280, 64)      3136      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 140, 140, 64)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 140, 140, 64)      65600     
                                                                 
 conv2d_2 (Conv2D)           (None, 140, 140, 64)      65600     
                                                                 
 conv2d_3 (Conv2D)           (None, 140, 140, 64)      65600     
                                                             

#### 2) 학습
* EarlyStopping 설정하고 학습시키기

In [ ]:
hist3 = model3.fit(x_train_sc, y_train, validation_data=(x_val_sc, y_val), epochs=10000, verbose=1, callbacks=[es])

Epoch 1/10000
14/14 [==============================] - 6s 398ms/step - loss: 0.6107 - accuracy: 0.7598 - val_loss: 0.6288 - val_accuracy: 0.6636
Epoch 2/10000
14/14 [==============================] - 5s 341ms/step - loss: 0.4514 - accuracy: 0.8152 - val_loss: 0.7514 - val_accuracy: 0.5545
Epoch 3/10000
14/14 [==============================] - 5s 346ms/step - loss: 0.4558 - accuracy: 0.8453 - val_loss: 0.8559 - val_accuracy: 0.5182
Epoch 4/10000
14/14 [==============================] - 5s 343ms/step - loss: 0.4404 - accuracy: 0.8176 - val_loss: 0.7917 - val_accuracy: 0.5818
Epoch 5/10000
14/14 [==============================] - 5s 371ms/step - loss: 0.3611 - accuracy: 0.8614 - val_loss: 0.7723 - val_accuracy: 0.5818
Epoch 6/10000
14/14 [==============================] - 5s 371ms/step - loss: 0.3616 - accuracy: 0.8453 - val_loss: 2.8859 - val_accuracy: 0.5455
Epoch 7/10000
14/14 [==============================] - 5s 347ms/step - loss: 0.3764 - accuracy: 0.8591 - val_loss: 0.6723 - val_ac

#### 3) test set으로 예측하고 평가하기
* 평가는 confusion_matrix, classification_report 활용

In [ ]:
y_pred3 = model3.predict(x_test_mi)
y_pred_new3 = y_pred3.argmax(axis=1)

2/2 [==============================] - 0s 108ms/step


In [ ]:
print(f'{classification_report(y_test_new, y_pred_new3)}')

              precision    recall  f1-score   support

           0       0.73      0.77      0.75        31
           1       0.76      0.71      0.73        31

    accuracy                           0.74        62
   macro avg       0.74      0.74      0.74        62
weighted avg       0.74      0.74      0.74        62



### (6)모델4

In [ ]:
from keras.layers import Input, Conv2D, MaxPooling2D, Activation, BatchNormalization, Add, Flatten, Dense
from keras.models import Model

def residual_block(x, filters, strides=(1,1), shortcut=None):
    shortcut = x
    x = Conv2D(filters, kernel_size=(3,3), strides=strides, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(filters, kernel_size=(3,3), strides=(1,1), padding='same')(x)
    x = BatchNormalization()(x)
    x = Add()([x, shortcut])
    x = Activation('relu')(x)
    return x

def resnet(input_shape, n_classes):
    input = Input(shape=input_shape)
    
    x = Conv2D(64, kernel_size=(7,7), strides=(2,2), padding='same')(input)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='same')(x)
    
    x = residual_block(x, filters=64, strides=(1,1))
    x = residual_block(x, filters=64, strides=(1,1))
    x = residual_block(x, filters=64, strides=(1,1))
    
    x = residual_block(x, filters=128, strides=(2,2))
    x = residual_block(x, filters=128, strides=(1,1))
    x = residual_block(x, filters=128, strides=(1,1))
    x = residual_block(x, filters=128, strides=(1,1))
    
    x = residual_block(x, filters=256, strides=(2,2))
    x = residual_block(x, filters=256, strides=(1,1))
    x = residual_block(x, filters=256, strides=(1,1))
    x = residual_block(x, filters=256, strides=(1,1))
    x = residual_block(x, filters=256, strides=(1,1))
    x = residual_block(x, filters=256, strides=(1,1))
    
    x = residual_block(x, filters=512, strides=(2,2))
    x = residual_block(x, filters=512, strides=(1,1))
    x = residual_block(x, filters=512, strides=(1,1))
    
    x = Flatten()(x)
    x = Dense(n_classes, activation='softmax')(x)
    
    model = Model(inputs=input, outputs=x)
    
    return model

## 4.모델링 II
* **세부요구사항**
    - 성능을 높이기 위해서 다음의 두가지를 시도해 봅시다.
        - Data Augmentation을 통해 데이터를 증가 시킵니다.
            - ImageDataGenerator를 사용합니다.
        - 사전 학습된 모델(Transfer Learning)을 가져다 사용해 봅시다.
            - VGG16(이미지넷)을 사용해 봅시다.

### (1) Data Augmentation
- **세부요구사항**
    * 모델 학습에 이용할 이미지 데이터를 증강시키세요.
    * Keras의 ImageDataGenerator를 이용
        - [ImageDataGenerator document](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator)

    * image generator를 이용하여 학습
        * 모델 구조는 이미 생성한 1,2,3 중 하나를 선택하여 학습


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
img_size = 280 ## 사이즈 조정 가능

train_path = dataset_path+'Car_Images_train/'
valid_path = dataset_path+'Car_Images_valid/'

#### 1) ImageGenerator 생성
* ImageDataGenerator 함수 사용
    * 주요 옵션
        * rotation_range: 무작위 회전을 적용할 각도 범위
        * zoom_range: 무작위 줌을 적용할 범위 [1-zoom_range, 1+zoom_range]
        * horizontal_flip: 무작위 좌우반전을 적용할지 여부
        * vertical_flip: 무작위 상하반전을 적용할지 여부
        * rescale: 텐서의 모든 값을 rescale 값으로 나누어줌 (이 경우에는 255로 나누어서 0~1사이의 값으로 변경)

In [ ]:
train_datagen = 

valid_datagen = 


#### 2) 경로로 부터 이미지 불러 올 준비
* .flow_from_directory 이용
    * 디렉토리에서 이미지를 가져와서 데이터 증강을 적용하고 batch 단위로 제공하는 generator를 생성합니다.
    * 이미지를 불러올 때 target_size로 크기를 맞추고, 
    * class_mode로 이진 분류(binary)를 수행하도록 지정합니다.


In [ ]:
train_generator = 

valid_generator = 


#### 3) 학습
- **세부요구사항**
    - Conv2D, MaxPooling2D, Flatten, Dense 레이어들을 이용하여 모델을 설계
    - 학습시 train_generator 이용. 
    - validation_data = valid_generator 지정
    - Early Stopping 적용
    - 평가시, confusion matrix, accuracy, recall, precision, f1 score 등을 이용하시오.

* 구조 설계

* 학습
    * EarlyStopping 설정하기
    * 학습 데이터에 train_generator, validation_data=valid_generator 사용

#### 4) 성능 평가
* 평가는 confusion_matrix, classification_report 활용

### (2) Transfer Learning
- **세부요구사항**
    * VGG16 모델은 1000개의 클래스를 분류하는 데 사용된 ImageNet 데이터셋을 기반으로 사전 학습된 가중치를 가지고 있습니다. 
        * 따라서 이 모델은 이미지 분류 문제에 대한 높은 성능을 보입니다.
        * 이 모델은 보통 전이학습(transfer learning)에서 기본적으로 사용되며, 특히 대규모 데이터셋이 없을 때는 기본 모델로 사용되어 fine-tuning을 수행합니다.
    * VGG16 함수로 부터 base_model 저장


In [ ]:
from tensorflow.keras.applications import VGG16

#### 1) VGG16 불러와서 저장하기
* include_top=False로 설정하여 분류기를 제외하고 미리 학습된 가중치 imagenet을 로드합니다.
* .trainable을 True로 설정하여 모델의 모든 레이어들이 fine-tuning에 대해 업데이트되도록 합니다.


In [ ]:
base_model = VGG16(                 )




#### 2) VGG16과 연결한 구조 설계
* VGG16을 불러와서 Flatten, Dense 등으로 레이어 연결하기

#### 3) 학습
- **세부요구사항**
    - 모델 학습 과정에 알맞은 보조 지표를 사용하세요.
    - 데이터
        * Image Generator를 연결하거나
        * 기존 train, validation 셋을 이용해도 됩니다.
        - Early Stopping을 반드시 사용하세요.
        - 최적의 가중치를 모델에 적용하세요.

#### 4) 성능 평가